In [ ]:
import whisperx

# identify word


In [ ]:
device = "cpu"
audio_file = "input.mp4"
batch_size = 32  # reduce if low on GPU mem
compute_type = "float32"  # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"])  # before alignment

# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(
    language_code=result["language"], device=device
)
result = whisperx.align(
    result["segments"], model_a, metadata, audio, device, return_char_alignments=False
)

In [6]:
result.keys()

dict_keys(['segments', 'word_segments'])

In [3]:
result

{'segments': [{'start': 0.049,
   'end': 1.194,
   'text': ' Thanks for trying EditFrame.',
   'words': [{'word': 'Thanks', 'start': 0.049, 'end': 0.25, 'score': 0.89},
    {'word': 'for', 'start': 0.27, 'end': 0.39, 'score': 0.776},
    {'word': 'trying', 'start': 0.431, 'end': 0.692, 'score': 0.823},
    {'word': 'EditFrame.', 'start': 0.732, 'end': 1.194, 'score': 0.734}]},
  {'start': 1.896,
   'end': 4.707,
   'text': 'This is an audio file we created for our documentation.',
   'words': [{'word': 'This', 'start': 1.896, 'end': 2.017, 'score': 0.997},
    {'word': 'is', 'start': 2.077, 'end': 2.137, 'score': 0.978},
    {'word': 'an', 'start': 2.177, 'end': 2.238, 'score': 0.954},
    {'word': 'audio', 'start': 2.378, 'end': 2.679, 'score': 0.996},
    {'word': 'file', 'start': 2.719, 'end': 3.061, 'score': 0.688},
    {'word': 'we', 'start': 3.081, 'end': 3.201, 'score': 0.501},
    {'word': 'created', 'start': 3.261, 'end': 3.603, 'score': 0.848},
    {'word': 'for', 'start': 3.

In [4]:
result["segments"]

[{'start': 0.049,
  'end': 1.194,
  'text': ' Thanks for trying EditFrame.',
  'words': [{'word': 'Thanks', 'start': 0.049, 'end': 0.25, 'score': 0.89},
   {'word': 'for', 'start': 0.27, 'end': 0.39, 'score': 0.776},
   {'word': 'trying', 'start': 0.431, 'end': 0.692, 'score': 0.823},
   {'word': 'EditFrame.', 'start': 0.732, 'end': 1.194, 'score': 0.734}]},
 {'start': 1.896,
  'end': 4.707,
  'text': 'This is an audio file we created for our documentation.',
  'words': [{'word': 'This', 'start': 1.896, 'end': 2.017, 'score': 0.997},
   {'word': 'is', 'start': 2.077, 'end': 2.137, 'score': 0.978},
   {'word': 'an', 'start': 2.177, 'end': 2.238, 'score': 0.954},
   {'word': 'audio', 'start': 2.378, 'end': 2.679, 'score': 0.996},
   {'word': 'file', 'start': 2.719, 'end': 3.061, 'score': 0.688},
   {'word': 'we', 'start': 3.081, 'end': 3.201, 'score': 0.501},
   {'word': 'created', 'start': 3.261, 'end': 3.603, 'score': 0.848},
   {'word': 'for', 'start': 3.643, 'end': 3.743, 'score': 0.

# write identify word to srt file


In [5]:
from datetime import timedelta
import os


def write_segments_to_srt(segments):
    for index, segment in enumerate(segments):
        startTime = str(0) + str(timedelta(seconds=int(segment["start"]))) + ",000"
        endTime = str(0) + str(timedelta(seconds=int(segment["end"]))) + ",000"
        text = segment["text"]
        print(text)
        segment = f"{index + 1}\n{startTime} --> {endTime}\n{text[1:] if text[0] == ' ' else text}\n\n"

        srtFilename = os.path.join(f"subtitles.srt")
        with open(srtFilename, "a", encoding="utf-8") as srtFile:
            srtFile.write(segment)


write_segments_to_srt(result["segments"])

 Thanks for trying EditFrame.
This is an audio file we created for our documentation.
We hope you enjoy it.
